In [1]:
from src.resources.integration.fparser_python import *
from src.resources.integration.fparser import *
from src.flasking.sending_windows import *
from src.langchain_flow.agents.segmenter_agent import *
from src.langchain_flow.agents.enhanced_ad_agent import *
from src.langchain_flow.agents.models_agent import *
config = {"configurable" : {"thread_id" : 1}}

Trying OLLAMA model 14b
OLLAMA model found


In [2]:
with open("./CRAM_Plan_Executive.txt", 'r') as f:
    context = f.read()

In [3]:
summary = parse_summary_data(context)

In [4]:
segments = parse_segment_data(context)

In [5]:
segments

[{'instruction': 'In Segment 1, the person begins preparing for a task by gathering a box of Cheez-It crackers, a red bowl, a red cup, and a bottle of mustard, likely to create a snack or dish.',
  'action_designator': {'action_designator': {'action_frame': {'id': 'act_seg_001',
     'type': 'gathering',
     'agent': 'human',
     'goal': 'gather_supplies',
     'temporal_phase': 'in_progress'},
    'manipulation': {'right_hand': {'role': 'executing',
      'action': 'pick_up',
      'grasp': {'type': 'precision',
       'contact_points': 'index_finger',
       'holding_type': 'object_grip'},
      'rotation_vector': [0.0, 0.0, 0.0, 1.0],
      'object': {'id': 'crackers_01',
       'name': 'crackers',
       'shape': 'flat',
       'handle': False,
       'weight': 0.2,
       'affordances': ['graspable', 'container_open'],
       'rotation_vector': [0.0, 0.0, 0.0, 1.0]}},
     'left_hand': {'role': 'supporting',
      'action': 'hold',
      'grasp': {'type': 'enclosing',
       'co

In [6]:
for seg in segments:
    print("📡 Sending test data to 1st window...")
    send_data_to_window(1, seg)

📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Win

In [7]:
summary_str = []
for k,v in  summary[0].items():
    summary_str.append(v)

In [8]:
final_state = workflow.invoke(initial_state(summary_str))
final_parse = final_state["final_instructions"]
final_parse_str = "".join(final_parse)
final_parse = [ele for ele in final_parse_str.split(".") if ele]

Task Objective: The person is assembling a snack by pouring Cheez-It crackers into a bowl, preparing to combine them with mustard and a cup as part of a meal setup.

Structured Activities: [{'intent': 'gather', 'object': 'a box of Cheez-It crackers, a red bowl, a red cup, and a bottle of mustard', 'target': None, 'tool': None, 'location': None, 'modifier': None, 'quantity': None, 'duration': None}, {'intent': 'open', 'object': 'box of Cheez-It crackers', 'target': None, 'tool': None, 'location': None, 'modifier': None, 'quantity': None, 'duration': None}, {'intent': 'pick up', 'object': 'box of Cheez-It crackers', 'target': None, 'tool': None, 'location': 'table', 'modifier': None, 'quantity': None, 'duration': None}, {'intent': 'pour', 'object': 'Cheez-It crackers', 'target': 'red bowl', 'tool': 'box', 'location': None, 'modifier': 'grasping-and-moving', 'quantity': 'a box', 'duration': 'not specified'}, {'intent': 'pour', 'object': 'Cheez-It crackers', 'target': 'red bowl', 'tool': N

In [9]:
final_parse

['Pick up the box of Cheez-It crackers',
 'Open the box of Cheez-It crackers',
 'Pour the Cheez-It crackers from the box into the red bowl',
 'Pour the Cheez-It crackers evenly into the bowl']

In [10]:
for par in final_parse:
    if par:
        print("📡 Sending test data to 2nd window...")
        send_data_to_window(2, par.strip())

📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}


In [11]:
graph_output = []
for fp in final_parse:
    # instruction = fp['instruction']
    # ad = fp['action_designator']
    print("Invoking Model")
    out = enhanced_ad_graph.invoke({'instruction': fp, 'context': ""})
    graph_output.append(out)

for out in graph_output:
    print("<UNK> Sending test data to 3rd window...")
    new_out = {}
    try:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = json.loads(out['action_core_attributes'])
        new_out['enriched_action_core_attributes'] = json.loads(out['enriched_action_core_attributes'])
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output normally")
    except:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = out['action_core_attributes']
        new_out['enriched_action_core_attributes'] = out['enriched_action_core_attributes']
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output with strings")

    send_data_to_window(3, new_out)

Invoking Model
INSIDE ACTION NODE
type={'action': 'PickingUp'}
['PickingUp']
INSIDE CRAM NODE
cram_plan (perform (an action (type grab-object) (an object (type {obj_to_be_grabbed}){obj_to_be_grabbed_props}) (source (a location (on {location}){location_props}))))
----------
{"obj_to_be_grabbed": "the box of Cheez-It crackers", "action_verb": "Pick up", "location": null}
----------
fCONTEXT 
{
  "action_verb": "Pick up",
  "obj_to_be_grabbed": "box of Cheez-It crackers",
  "obj_to_be_grabbed_props": {
    "shape": "rectangular",
    "material": "cardboard",
    "texture": "smooth",
    "pattern": "printed",
    "cleanliness": "clean",
    "intactness": "intact"
  },
  "location": null
}
----------
(perform (an action (type grab-object) (an object (type box of Cheez-It crackers) (shape rectangular) (material cardboard) (texture smooth) (pattern printed) (cleanliness clean) (intactness intact))))
----------
Invoking Model
INSIDE ACTION NODE
type={'action': 'Opening'}
['Opening']
INSIDE CRA

In [12]:
framenet_model = ""
flanagan = ""

In [13]:
for out in graph_output[:1]:
    print("<UNK> Sending test data to 4th window...")
    instruction = out['instruction']
    action_core = out['action_core']
    action_core_attributes = out['action_core_attributes']
    enriched_action_core_attributes = json.loads(out['enriched_action_core_attributes'])
    cram_plan = out['cram_plan_response']
    final_graph_state = model_graph.invoke({"instruction": instruction,
                                            "action_core": action_core,
                                            "enriched_action_core_attributes": enriched_action_core_attributes,
                                            "cram_plan_response": cram_plan}, config=config, stream_mode="updates")
    flanagan = model_graph.get_state(config).values["flanagan"]
    framenet_model = model_graph.get_state(config).values["framenet_model"]
    new_out = {}
    try:
        flanagan_json = json.loads(flanagan)
        framenet_model_json = json.loads(framenet_model)
        print("Parsed models output normally")
        new_out = {
            "framenet": framenet_model_json,
            "flanagan": flanagan_json
        }
    except:
        print("Parsed models output with strings")
        new_out = {
            "framenet": framenet_model,
            "flanagan": flanagan
        }

    send_data_to_window(4, new_out)

<UNK> Sending test data to 4th window...
NEW FLANAGAN NODE STATE  {'messages': [], 'instruction': 'Pick up the box of Cheez-It crackers', 'action_core': 'PickingUp', 'enriched_action_core_attributes': {'action_verb': 'Pick up', 'obj_to_be_grabbed': 'box of Cheez-It crackers', 'obj_to_be_grabbed_props': {'shape': 'rectangular', 'material': 'cardboard', 'texture': 'smooth', 'pattern': 'printed', 'cleanliness': 'clean', 'intactness': 'intact'}, 'location': None}, 'cram_plan_response': '(perform (an action (type grab-object) (an object (type box of Cheez-It crackers) (shape rectangular) (material cardboard) (texture smooth) (pattern printed) (cleanliness clean) (intactness intact))))'}
INSIDE FRAMENET NODE
Parsed models output with strings
✅ Data sent to Window 4: {'status': 'success', 'window': 'window4'}


In [ ]:
print(flanagan)

In [ ]:
print(framenet_model)